<a href="https://colab.research.google.com/github/finardi/WatSpeed_LLM_foundation/blob/main/Module2%3A%20Lab_assignment_GPT_3_5_multi_document_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2 - Lab assignment - GPT-3.5 prompt engineering

This notebook presents an example of how to use the GPT-3.5-turbo model to perform multi-document question answering. The task consists of answering a question given a set of documents that may contain relevant information. In this lab assignment, we will focus on prompt engineering, which creates effective prompts to improve the model's performance.

We will work with the GPT-3.5-turbo model, a large-scale language model based on the GPT-3 architecture. This model has been fine-tuned on a wide range of natural language processing tasks and is known for its high performance.

In this notebook, we use the [Incomplete Information Reading Comprehension (IIRC)](https://allenai.org/data/iirc) dataset as a benchmark. The IIRC dataset contains questions that require the model to reason over multiple documents to generate a complete answer. It is a challenging dataset that requires the model to extract relevant information from multiple sources and synthesize it to produce an accurate answer.

We will focus on prompt engineering using the CoT approach to improve the model's performance on this task. CoT involves generating a reasoning paragraph that guides the model to focus on the most relevant information when answering the question. By providing the model with a well-crafted prompt, we can improve its ability to extract and synthesize information from multiple sources, leading to more accurate and complete answers, as demonstrated in [Visconde](https://arxiv.org/pdf/2212.09656.pdf).

**Assignment:** search for `TODO:` in the cells and write your code to accomplish the task.

# Installing required packages

In this example, we have to install `openai` and `tiktoken` libraries.

**`openai`**:

OpenAI is an artificial intelligence research laboratory consisting of the for-profit corporation OpenAI LP and its parent company, the non-profit OpenAI Inc. The OpenAI library is a powerful machine learning library that provides an easy-to-use interface to the OpenAI API. With this library, users can easily integrate OpenAI's state-of-the-art language models, including GPT-3, into their applications, and leverage the full power of these models to perform various natural language processing (NLP) tasks, such as language generation, classification, question-answering, and more.

**`tiktoken`**:

Tiktoken is an open-source BPE tokeniser developed by OpenAI that is used to split text strings into tokens. It is useful for models like GPT-3 that encode text into tokens. Tiktoken is designed to be highly efficient, capable of handling large amounts of text quickly and accurately.

In [1]:
!pip install -q openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 767.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.4 MB/s eta 0:00:00


# Downloading the data

To use the Incomplete Information Reading Comprehension (IIRC) dataset as a benchmark, we need to download the data. The IIRC dataset consists of a set of documents and associated questions. We can download the dataset test set using the following code:



In [2]:
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json

--2023-05-14 12:58:20--  https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 52.92.136.178, 3.5.82.161, 52.92.152.74, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|52.92.136.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874825 (2.7M) [application/json]
Saving to: ‘iirc_test.json’

iirc_test.json      100%[===================>]   2.74M  4.70MB/s    in 0.6s    

2023-05-14 12:58:21 (4.70 MB/s) - ‘iirc_test.json’ saved [2874825/2874825]



Let's load the data and see what it looks like. 

We are using the IIRC dataset, and we have imported the JSON library to read the test set file. We loaded the first example from the test set, which is a dictionary with keys 'questions', 'text', 'links', and 'title'. 

The 'questions' key contains a list of dictionaries with keys 'question', 'context', 'answer', and 'question_links'. The 'text' key contains the text that may contain relevant information for answering the questions. The 'links' key is a list of dictionaries with keys 'target' and 'indices', indicating the hyperlink target and the position of the hyperlink in the text. The 'title' key contains the title of the document. 

In this particular example, we can see that the question is "What is Zeus known for in Greek mythology?" and the answer is "being the sky and thunder god". The context contains three passages containing the text that may provide additional information.

In [3]:
import json

test_set = json.load(open('iirc_test.json','r'))

test_set[0]

{'questions': [{'answer': {'type': 'span',
    'answer_spans': [{'text': 'sky and thunder god',
      'passage': 'zeus',
      'type': 'answer',
      'start': 83,
      'end': 102}]},
   'question': 'What is Zeus know for in Greek mythology?',
   'context': [{'text': 'he Palici the sons of Zeus',
     'passage': 'main',
     'indices': [684, 710]},
    {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
    {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
     'passage': 'Zeus',
     'indices': [0, 110]}],
   'question_links': ['Greek mythology', 'Zeus']}],
 'text': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia 

## Data Preparation

When working with the IIRC dataset, preparing the data before using it to evaluate models is necessary. Additionally, it is important to carefully choose how many questions to evaluate, as using GPT from OpenAI API can be costly. The data preparation process involves iterating through the test set,
extracting relevant information such as the question, documents containing relevant passages, and the true answer.

This is achieved by parsing the original JSON format and mapping it into a format that can be used to evaluate models. 

In the code below, the preparation process is limited to a maximum of 50 questions due to the cost associated with using the OpenAI API. It involves iterating through each question and its corresponding context, creating a list of documents containing relevant information, and extracting the true answer based on its type. By preparing the data in this way, it can be more easily fed into GPT models, and compare the results. 

We use a limited number of questions to reduce the costs of using OpenAPI API.

In [35]:
max_questions = 50 # @param

test_set_questions = []
i = 0 
while len(test_set_questions) < max_questions:

  item = test_set[i]
  for question in item['questions']:
    documents = []    
    for doc in question['context']:
      documents.append({
          "title": doc['passage'] if doc['passage'] != "main" else item['title'],
          "content": doc['text']
      })
    true_answer = ""
    if question['answer']['type'] == "span":
      true_answer = ", ".join([a['text'] for a in question['answer']["answer_spans"]])
    elif question['answer']['type'] == "value":
        true_answer = "{0} {1}".format(question['answer']['answer_value'],question['answer']['answer_unit'])
    elif question['answer']['type'] == "binary":
        true_answer = question['answer']['answer_value']
    elif question['answer']['type'] == "none":
        true_answer = "Not enough information."
    test_set_questions.append({
        "question": question['question'],
        "documents": documents,
        "answer": true_answer
    })
    i+=1


The resulting prepared dataset is a dictionary with four keys. The "**`question`**" key contains the actual question to be answered, which in this case is "What is Zeus known for in Greek mythology?". The "**`documents`**" key is a list containing information that may be relevant to answering the question. Each document is a dictionary with a "title" key and a "content" key. The "title" key gives the name of the source of the information, while the "content" key provides the actual text of the source. In this case, there are three documents, all related to the topic of Greek mythology and Zeus. The "**`answer`**" key contains the correct answer to the question, which is "sky and thunder god".



In [36]:
test_set_questions[0]

{'question': 'What is Zeus know for in Greek mythology?',
 'documents': [{'title': 'Palici', 'content': 'he Palici the sons of Zeus'},
  {'title': 'Palici', 'content': 'in Greek mythology'},
  {'title': 'Zeus',
   'content': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion'}],
 'answer': 'sky and thunder god'}

# Using OpenAI API

To use OpenAI API, we need to set our API key and import the OpenAI module. In the given code, we have the `OPENAI_KEY` variable which we can set to our OpenAI API key. After that, we can use the `openai.api_key` method to set the API key for our session.

The function `generate_chat` takes in a list of messages and generates a response using the OpenAI Chat API. The `model` parameter specifies which model to use for generating the response. In the given code, we have used the `gpt-3.5-turbo` model. However, we can also use `gpt-4`.

The function `generate` takes in a prompt and generates a completion using the OpenAI Completion API. In the given code, we have used the `text-davinci-003` engine, which is a powerful language model that can generate human-like responses to text prompts. We can also set various parameters such as `max_tokens`, `temperature`, `top_p`, `frequency_penalty`, and `presence_penalty` to control the behavior of the model. 

**`top_p`**: This parameter controls the "nucleus sampling" algorithm used by the language model to generate text. It determines the probability threshold for including words in the generated text. Specifically, the algorithm starts with the most probable words and keeps adding them until their cumulative probability reaches `top_p`. This allows the model to generate more diverse and varied responses. The default value is 1, which means the algorithm always includes the most probable words.

**`frequency_penalty`**: This parameter discourages the model from repeating the same words or phrases too often in the generated text. The higher the value, the more the model is penalized for repeating itself. The default value is 0, which means no penalty is applied.

**`presence_penalty`**: This parameter encourages the model to include words or phrases from the prompt or context in the generated text. The higher the value, the more the model is penalized for generating words or phrases that are not present in the prompt or context. The default value is 0, which means no penalty is applied.

**`temperature`**: Temperature is a hyperparameter in natural language processing (NLP) models like GPT-3 that controls the degree of randomness in the output of the model. A higher temperature value leads to more diverse and unpredictable responses, while a lower temperature value results in more conservative and predictable responses.

**IMPORTANT:** It's important to note that there are costs associated with using the OpenAI API, so we need to choose the appropriate model and set the parameters carefully to avoid unnecessary expenses. The `gpt-3.5-turbo` model used in `generate_chat` is a cheaper option compared to `text-davinci-003`.

In [37]:
import os
import openai

OPENAI_KEY = "" # @param set your OpenAI API key here

openai.api_key = OPENAI_KEY

def generate_chat(messages,model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response["choices"][0]['message']['content']


def generate(prompt,max_tokens=128, temperature=0):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response["choices"][0]['text']

# Question Answering


In this section, we perform the question-answering step. Using the prepared dataset, we apply the GPT-3.5 model to generate answers to the questions provided in the test set. The goal is to evaluate the performance of the model in providing accurate answers to natural language questions. This is a crucial step in natural language processing, as it enables machines to understand and respond to human queries. We will use the OpenAI API to generate the answers and evaluate the results against the true answers provided in the test set.

In this example, we will use a 3-shot prompt with Chain-of-thought available [here](https://platform.openai.com/playground/p/bjGeM1rNaxzm6GANTAwNXz1l).

In the following sub-sections, we'll review each part of this prompt. 

## Instruction

To perform the question answering task, GPT-3 and GPT-3.5 need to be instructed on the task they have to perform. The instruction is defined in the code below.

This instruction specifies that the model should use the provided documents to generate an answer to the question and provide an explanation for how it arrived at that answer. Additionally, the model should cite evidence from the documents to support its answer. If there is not enough information in the documents to answer the question, the model should indicate that with the phrase "not enough information". The expected format for the model's response is also defined in the instruction.

```
Expected Format:
Explanation: <>
Answer: <>
```

Defining the expected format is important because it provides a clear guideline for the model to follow when generating its response. By providing a clear template for the output, we can ensure that the model outputs information in a consistent and organized manner. This makes it easier for us to post-process the model's generation and extract the target information.

[Best practices for prompt engineering with OpenAI API
](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api)

In [38]:
instruction = """For each example, use the documents to provide an answer to the question and cite 
evidence from the documents to support the answer. If there is not enough information in the documents 
to answer the question, then state "not enough information".
Expected Format:
Explanation: <>
Answer: <>
"""

## Few-shot examples

The second part of the prompt consists of the few-shot examples. The few-shot examples are crucial in this approach because they enable the model to learn how to perform the task with only a small number of examples. The few-shot examples allow the model to "understand" how to answer questions based on a small set of examples, which makes it more robust and capable of answering a wider range of questions.

In the code provided below, we define three multi-doc QA examples that we will use as few-shot examples. These examples will serve as inputs to the OpenAI API to generate responses for the given questions. We will provide the context documents and the question as the prompt to the API, and it will generate the answer for us. By using multi-doc examples, we allow the API to consider multiple sources of information to provide us with the best answer possible. These few-shot examples allow us to see how well the OpenAI API can perform on previously unseen questions that are similar to the examples we provide.

In [39]:
examples = [
    {
        "documents": [
            {"title": "Fred J. Shields", "content": "Ollie Murphy's first-half goal gave 'the Royals' a huge boost at half-time."},
            {"title": "Ollie Murphy", "content": "He plays club football for Carnaross"},
            {"title": "Ollie Murphy", "content": "He came to national prominence in 1999 when he was one of Meath's best player's"}
        ],
        "question": "Did Ollie Murphy play for any teams other than 'the Royals'?",
        "explanation": "Document 1 says that Ollie Murphy's first-half goal gave 'the Royals' a huge boost at half-time. However, this does not necessarily mean that Ollie only played for 'the Royals'. Document 2 states that Ollie plays club football for Carnaross. This suggests that Ollie may have played for other teams in addition to 'the Royals'. Document 3 says that Ollie came to national prominence in 1999 when he was one of Meath's best players. This also suggests that Ollie has played for other teams. Therefore, it is likely that Ollie has played for teams other than 'the Royals'.",
        "answer": "yes"
    },
    {
        "documents": [
            {"title": "Don Rendell", "content": "The club played in the Brunei Premier League in the early 2000s, winning the league title in 2002 and 2004."}
        ],
        "question": "What club came in second at the 2004 Brunei Premier League?",
        "explanation": "Despite the document 1 being about the premier league and saying who won it in 2004, there is not enough information to determine which club came in second place during that year.",
        "answer": "not enough information."
    },
    {
        "documents": [
            {"title": "Stacy Compton", "content": "Despicable Me, the first film in the series, and the first film from Illumination, was released on July 9, 2010."},
            {"title": "Miranda Cosgrove", "content": "Cosgrove's first television appearance (aside from commercials) was in 2001 as the voice of young Lana Lang in the pilot episode of Smallville."},
            {"title": "Miranda Cosgrove", "content": "In 2004, Cosgrove soon landed her first major role in a television series when she was awarded a main role in the Nickelodeon series Drake & Josh"},
            {"title": "Miranda Cosgrove", "content": "Also in 2004, Cosgrove guest-starred in a special episode of the animated series What's New, Scooby-Doo?, as well as guest-starring in a season five episode of Grounded For Life"},
            {"title": "Miranda Cosgrove", "content": "The television series, which aired on Disney, is a spin-off of the original film, Lilo & Stitch"},
            {"title": "Miranda Cosgrove", "content": "The first of these appearances was in Zoey 101. Cosgrove later guest starred on an episode of Unfabulous,"},
            {"title": "Miranda Cosgrove", "content": "However, Cosgrove was already in the works of starring in her own sitcom, titled iCarly, released on September 8, 2007."}
        ],
        "question": "How many TV shows had Miranda Cosgrove been featured in by the year Despicable Me was released?",
        "explanation": "According to document 1, Despicable Me was released on July 9, 2010.Document 2 states that Cosgrove's first television appearance was in 2001 as the voice of young Lana Lang in the pilot episode of Smallville.Document 3 says that, in 2004, Cosgrove landed her first major role in a television series when she was awarded a main role in the Nickelodeon series Drake & Josh.Document 4 states that, also in 2004, Cosgrove guest-starred in a special episode of the animated series What's New, Scooby-Doo?, as well as guest-starring in a season five episode of Grounded For Life.Document 5 says that the television series, which aired on Disney, is a spin-off of the original film, Lilo & Stitch.Document 6 states that the first of these appearances was in Zoey 101. Cosgrove later guest starred on an episode of Unfabulous.Document 7 says that, however, Cosgrove was already in the works of starring in her own sitcom, titled iCarly, released on September 8, 2007.Therefore, Miranda Cosgrove had been featured in 8 TV shows by the year Despicable Me was released.",
        "answer": "8 TV shows"
    }
]

## Defining post-processing function

The next code cells aim to answer the multi-document question using OpenAI API to consume two models: InstructGPT `text-davinci-003` and ChatGPT `gpt-3.5-turbo`.

Considering our example prompt, the next steps consist of appending the target example to the end of the prompt.

First, we have to define an "**util**" function responsible for extracting the explanation and the answer written by the model. The code below defines a function named `extract_explanation_and_answer`, which takes a string `text` as input. This function uses regular expressions to search the text for an "Explanation" and "Answer" section, which are expected to be formatted as "Explanation: \<explanation text\>\n" and "Answer: \<answer text\>". It then extracts the text after the "Explanation:" label and before the newline character as the explanation, and the text after the "Answer:" label as the answer.

In [68]:
a = """ 
Explanation: Document 1 mentions that Calvin Mackie was featured on HBO as a commentator on Spike Lee's documentary on the Katrina disaster When The Levees Broke: A Requiem in Four Parts. Document 2 states that When the Levees Broke: A Requiem in Four Acts is a 2006 documentary film directed by Spike Lee about the devastation of New Orleans, Louisiana following the failure of the levees during Hurricane Katrina. It was filmed in late August and early September 2005, and premiered at the New Orleans Arena on August 16, 2006. Therefore, Spike Lee debuted his documentary When the Levees Broke;\nAnswer: August and September of 2005"""
extract_explanation_and_answer(a)    

("Document 1 mentions that Calvin Mackie was featured on HBO as a commentator on Spike Lee's documentary on the Katrina disaster When The Levees Broke: A Requiem in Four Parts. Document 2 states that When the Levees Broke: A Requiem in Four Acts is a 2006 documentary film directed by Spike Lee about the devastation of New Orleans, Louisiana following the failure of the levees during Hurricane Katrina. It was filmed in late August and early September 2005, and premiered at the New Orleans Arena on August 16, 2006. Therefore, Spike Lee debuted his documentary When the Levees Broke;",
 'August and September of 2005')

In [85]:
chatGPT_output[-1]

"\nExplanation: Document 1 mentions that Calvin Mackie was featured on HBO as a commentator on Spike Lee's documentary on the Katrina disaster When The Levees Broke: A Requiem in Four Parts. Document 2 states that When the Levees Broke: A Requiem in Four Acts is a 2006 documentary film directed by Spike Lee about the devastation of New Orleans, Louisiana following the failure of the levees during Hurricane Katrina. It was filmed in late August and early September 2005, and premiered at the New Orleans Arena on August 16, 2006. Therefore, Spike Lee debuted his documentary When the Levees Broke;"

In [94]:
import re

def extract_explanation_and_answer(text):
    # Define regular expressions to match the explanation and answer
    explanation_regex = r"Explanation:\s(.+?)\n"
    answer_regex = r"Answer:\s(.+)"

    try:
        # find the explanation and answer in the text
        explanation = re.search(explanation_regex, text).group(1)
        answer = re.search(answer_regex, text).group(1)
        return explanation, answer
    
    except:
        # sometimes gpt3.5-turbo do not return the answer
        explanation = text
        answer = 'Answer: Not enough information.'
        return explanation, answer

## Using InstructGPT (text-davinci-003)

The below code defines a function `gpt3_qa` which performs the multi-document question answering task using the InstructGPT model (text-davinci-003). The function takes four arguments - `question`, `documents`, `examples`, and `get_prompt` (optional). 

`question` is a string that contains the question for which the answer is required.

`documents` is a list of dictionaries, where each dictionary represents a document and has two keys - `title` and `content`.

`examples` is a list of dictionaries, where each dictionary represents a few-shot example and has three keys - `question`, `explanation`, and `answer`. Additionally, each dictionary also has a key `documents`, which is a list of dictionaries representing the documents relevant to that example.

`get_prompt` is an optional boolean argument. If set to `True`, the function returns the generated prompt instead of the extracted explanation and answer.

The function appends the target question and documents to the end of the instruction prompt, and then generates the response using the InstructGPT model. The generated response is then passed through the `extract_explanation_and_answer` function to extract the explanation and answer. 

Finally, the function returns the extracted explanation and answer. If `get_prompt` is set to `True`, the function returns the generated prompt instead of the extracted explanation and answer.

In [53]:
def gpt3_qa(question, documents, examples, get_prompt=False):
  
  prompt = instruction # Write the instruction

  # Add the few-shot examples
  for i, example in enumerate(examples):
    prompt += f"Example {i+1}:\n"
    for j, doc in enumerate(example["documents"]):
      prompt += f"[Document {j+1}]: Title: {doc['title']}. Content: {doc['content']}\n"
    prompt += f"Question: {example['question']}\nExplanation: {example['explanation']}\nAnswer: {example['answer']}\n##\n"
  
  # Add target example
  prompt += f"Example {i+2}:\n"
  for k, doc in enumerate(documents):
    prompt += f"[Document {k+1}]: Title: {doc['title']}. Content: {doc['content']}\n"
  prompt += f"Question: {question}"

  if get_prompt: # return the formated prompt
    return prompt

  res = generate(prompt) # perform API call

  return extract_explanation_and_answer(res) # post-process the output and return exaplation and answer

## Using ChatGPT (gpt-3.5-turbo)

The `chatgpt_qa` function is used to perform multi-document question answering using the ChatGPT `gpt-3.5-turbo` model. Similar to the InstructGPT function, the `chatgpt_qa` function requires a question and a list of documents as inputs. 

When using the `gpt-3.5-turbo` model from OpenAI API for a conversational task, it is necessary to define three roles: `system`, `user`, and `assistant`. These roles are used to create a conversation flow between the user and the model, where the `system` role represents the model, the `user` role represents the user input, and the `assistant` role represents the model's response. [See more](https://platform.openai.com/docs/guides/chat)


To prompt the model, the function uses a list of `messages`, which includes a `"role"` key to specify whether the message is from the `"user"` or the `"assistant"`. The first message in the list is a system message that includes the `instruction` on how the model should answer the question. 

The function also appends the few-shot examples to the `messages` list, with each example being split into two messages: one from the user with the example question and documents, and one from the assistant with the example explanation and answer. 

Finally, the function appends a message from the user with the target question and documents to the `messages` list. The `generate_chat` function is then called to perform the API call to the OpenAI API using the `messages` list as input. 

The output of the `generate_chat` function is then post-processed using the `extract_explanation_and_answer` function to extract the explanation and answer provided by the model.

In [54]:
#  TODO

# global variable (not ellegant, but effective in $$)

chatGPT_output = [] # to avoid re-run chatGPT

def chatgpt_qa(question, documents, examples, model="gpt-3.5-turbo", get_prompt=False, verbose=True):
    # Write the instruction
    prompt = instruction 

    # Add the few-shot examples
    for i, example in enumerate(examples):
        prompt += f"Example {i+1}:\n"
        for j, doc in enumerate(example["documents"]):
            prompt += f"[Document {j+1}]: Title: {doc['title']}. Content: {doc['content']}\n"
        prompt += f"Question: {example['question']}\nExplanation: {example['explanation']}\nAnswer: {example['answer']}\n##\n"
    
    # Add target example
    prompt += f"Example {i+2}:\n"
    for k, doc in enumerate(documents):
        prompt += f"[Document {k+1}]: Title: {doc['title']}. Content: {doc['content']}\n"
    prompt += f"Question: {question}"
    
    # return the formated prompt
    if get_prompt: 
        return prompt
    
    # perform API call
    res = generate(prompt) 

    if verbose:
        print(res)
    
    chatGPT_output.append(res)
    # post-process the output and return exaplation and answer
    return extract_explanation_and_answer(res) 

## How much will it cost

Before running our experiments, it's important to know how much it will cost to use OpenAI API. The price of the API is based on the number of tokens used. Each request to the API costs a certain number of tokens, which can vary depending on the size of the model used, the length of the prompt, and other factors.

As of May 9, 2023, the costs of using OpenAI API are:

* GPT-4: \$0.03 /1k tokens
* GPT-3.5-turbo: \$0.002/1k tokens
* InstructGPT (text-davinci-003): \$0.02/1k tokens

[OpenAI API Pricing](https://openai.com/pricing)

The following code estimates the cost of running a few-shot question-answering experiment using OpenAI's GPT models. The estimation is based on the length of the input prompt and the expected length of the model output, which is set to 128 tokens in this case. The code uses the **`gpt3_qa`** function to generate prompts for each question in the test set, with a fixed number of examples (2 in this case) used for each question. The length of each prompt is then calculated using tokenizer loaded using **`tiktoken`** library. The final cost estimation takes into account the total length of all prompts, divided by 1000, and multiplied by the price per token of the selected model.



In [55]:
import numpy as np
import tiktoken

prompt_lengths = []
k_shot = 3 # @param [1,2,3]
expected_output_size = 128 # @param
models = ["text-davinci-003","gpt-3.5-turbo","gpt-4"]
for model in models:

    model_price = {"text-davinci-003": 0.02, "gpt-3.5-turbo": 0.002, "gpt-4": 0.03}[model]

    tokenizer = tiktoken.encoding_for_model(model)

    for question in test_set_questions:
        prompt = gpt3_qa(question['question'], question['documents'],examples[:k_shot], get_prompt=True)
        
        tokens = tokenizer.encode(prompt)

        prompt_lengths.append(len(tokens)+expected_output_size)

    price = (np.sum(prompt_lengths)/1000) * model_price
    print(f"This experiment will cost about U$ {price:.2f} using {model}")

This experiment will cost about U$ 1.31 using text-davinci-003
This experiment will cost about U$ 0.26 using gpt-3.5-turbo
This experiment will cost about U$ 5.94 using gpt-4


## Running

The code provided below is the main code to run the experiments. It uses the test questions set and loops through each question, retrieving the ground truth documents, the query question, and the pre-defined examples. Then, it uses the OpenAI API to retrieve the predicted answer and explanation to the given question using either the **`gpt3_qa`** or the **`chatgpt_qa`** function, depending on the chosen model. Afterward, it stores the predicted answer and explanation in the **`question`** dictionary. This process is repeated for each question in the test set. The **`tqdm`** library is used to display a progress bar that informs the current status of the loop. The **`k_shot`** parameter defines the number of documents the model can use to learn from.

❗ **Disclaimer** ❗: The user assumes entire responsibility over the expenses on using OpenAI API by running this example. Please be aware of the API usage and cost associated with each call, and use it at your own risk. Make sure to choose the correct model and set the correct parameters to avoid unnecessary costs. Also, running the cell multiple times may result in multiple API calls and higher costs.

In [102]:
from tqdm import tqdm
model = "gpt-3.5-turbo" # @param ["gpt-3.5-turbo","text-davinci-003","gpt-4"]
k_shot = 3 # @param [1,2,3]
for ix, question in enumerate(tqdm(test_set_questions)):
    if model in ["gpt-3.5-turbo", "gpt-4"]:
        explanation, answer = chatgpt_qa(question['question'], question['documents'], examples[:int(k_shot)])
    else:
        explanation, answer = gpt3_qa(question['question'], question['documents'], examples[:k_shot])
    question["explanation"] = explanation
    question["predicted_answer"] = answer

  2%|▏         | 1/52 [00:04<04:02,  4.76s/it]


Explanation: Document 1 states that the Palici are the sons of Zeus. Document 2 states that the Palici are from Greek mythology. Document 3 states that Zeus is the sky and thunder god in ancient Greek religion. Therefore, Zeus is known for being the sky and thunder god in Greek mythology.
Answer: being the sky and thunder god


  4%|▍         | 2/52 [00:11<04:44,  5.70s/it]


Explanation: Document 1 states that Messe became aide-de-camp to King Victor Emmanuel III from 1923 to 1927. Document 2 states that the First World War lasted from 28 July 1914 to 11 November 1918. Therefore, the First World War had been over for 5 years when Messe was named aide-de-camp.
Answer: 5 years


  6%|▌         | 3/52 [00:17<04:56,  6.06s/it]

...


Explanation: Document 1 mentions the Louisiana Recovery Authority (LRA), which was created to lead the state's rebuilding efforts following the catastrophic 2005 Hurricanes Katrina and Rita. Document 2 states that Hurricane Katrina was a Category 5 hurricane that made landfall on Florida and Louisiana in August 2005. Document 3 says that Rita formed near The Bahamas from a tropical wave on September 18, 2005. Therefore, Hurricane Katrina and Hurricane Rita hit in August and September of 2005, respectively.
Answer: August and September of 2005


 98%|█████████▊| 51/52 [05:53<00:06,  6.91s/it]


Explanation: Document 1 mentions that Calvin Mackie was featured on HBO as a commentator on Spike Lee's documentary on the Katrina disaster When The Levees Broke: A Requiem in Four Parts. Document 2 states that When the Levees Broke: A Requiem in Four Acts is a 2006 documentary film directed by Spike Lee about the devastation of New Orleans, Louisiana following the failure of the levees during Hurricane Katrina. It was filmed in late August and early September 2005, and premiered at the New Orleans Arena on August 16, 2006. Therefore, Spike Lee debuted his documentary When the Levees Broke;


100%|██████████| 52/52 [05:55<00:00,  6.84s/it]


Explanation: There is not enough information in the document to answer this question.
Answer: Not enough information.


# Evaluation

In this section, we evaluate the performance of our model by calculating the exact match score and the F1 score using bag of words. To accomplish this, we have defined some helper functions. 

The **`normalize_text`** function takes a text and normalizes it by converting it to lowercase and removing any non-alphanumeric characters. The **`get_tokens`** function tokenizes the text after normalization.

The **`exact_match`** function takes the predicted answer and the true answer and returns whether they match exactly after normalization. The **`f1_bag_of_words`** function takes the predicted answer and the true answer, tokenizes them, and calculates their F1 score using the bag of words approach. 

The bag of words approach is a technique used to measure the similarity between two sets of texts by counting the frequency of each word in both sets and then calculating their overlap.

In [103]:
import re
from collections import Counter

def normalize_text(text):
    """
    Helper function to normalize the text
    """
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]+", " ", text)
    return text.strip()

def get_tokens(text):
    """
    Helper function to tokenize text
    """
    text = normalize_text(text)
    return text.split()

def exact_match(pred_answer, true_answer):
    """
    Calculates the exact match score
    """
    return normalize_text(pred_answer) == normalize_text(true_answer)

def f1_bag_of_words(pred_answer, true_answer):
    """
    Calculates the F1 score using bag of words
    """
    pred_tokens = get_tokens(pred_answer)
    true_tokens = get_tokens(true_answer)

    pred_counter = Counter(pred_tokens)
    true_counter = Counter(true_tokens)

    common = pred_counter & true_counter
    num_same = sum(common.values())

    if num_same == 0:
        return 0

    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(true_tokens)
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

In the below code, we are evaluating the performance of the model for multi-document question answering by calculating the: Exact Match (EM) and F1 score using bag of words. 

The code iterates over each item in the validation dataset and calculates the F1 and EM scores using the **`f1_bag_of_words`** and **`exact_match`** functions defined earlier. The maximum score of F1 and EM is then taken for each item and appended to their respective lists, **`f1s`** and **`ems`**. 

The mean of the **`f1s`** and **`ems`** lists are then calculated using NumPy's **`np.mean`** function and assigned to **`mean_f1`** and **`mean_em`** variables, respectively. Finally, the average EM and F1 scores are printed using formatted string literals.

In [104]:
import numpy as np

f1s, ems = [], []
for question in test_set_questions:
  if "predicted_answer" in question:
    f1 = f1_bag_of_words(question["predicted_answer"],question["answer"])
    em = exact_match(question["predicted_answer"],question["answer"])
    f1s.append(f1)
    ems.append(em)

mean_em = np.mean(ems)
mean_f1 = np.mean(f1s)
print(f"Exact match: {mean_em:.3f}\nF1-bow: {mean_f1:.3f}")

Exact match: 0.788
F1-bow: 0.908


# Try your own questions

You can try multi-document QA using your own examples. Google for topics of your interest and provide the documents and the question as in the example below.

The documents used as an example below are the first paragraph from the pages "Pele" and "Steve Jobs" from Wikipedia. It is just a toy example. You can try more realistic ones.

In [105]:
documents = [
    {
        "title": "Pele",
        "content": "Edson Arantes do Nascimento (Brazilian Portuguese: [ˈɛdsõ aˈɾɐ̃tʃiz du nasiˈmẽtu]; 23 October 1940 – 29 December 2022), better known by his nickname Pelé (Portuguese pronunciation: [peˈlɛ]), was a Brazilian professional footballer who played as a forward. Widely regarded as one of the greatest players of all time, he was among the most successful and popular sports figures of the 20th century.[2][3] In 1999, he was named Athlete of the Century by the International Olympic Committee and was included in the Time list of the 100 most important people of the 20th century. In 2000, Pelé was voted World Player of the Century by the International Federation of Football History & Statistics (IFFHS) and was one of the two joint winners of the FIFA Player of the Century. His 1,279 goals in 1,363 games, which includes friendlies, is recognised as a Guinness World Record.[4]"
    },
    {
        "title": "Steve Jobs",
        "content": "Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American business magnate, inventor, and investor. He was the co-founder, chairman, and CEO of Apple; the chairman and majority shareholder of Pixar; a member of The Walt Disney Company's board of directors following its acquisition of Pixar; and the founder, chairman, and CEO of NeXT. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak."
    }
]

question = "How old were Pelé when Steve Jobs died?"

explanation, answer = chatgpt_qa(question, documents, examples)

print(f"Explanation: {explanation}\nAnswer: {answer}")


Explanation: Document 1 states that Pelé was born on October 23, 1940. Document 2 states that Steve Jobs died on October 5, 2011. Therefore, Pelé was 70 years old when Steve Jobs died.
Answer: 70 years old
Explanation: Document 1 states that Pelé was born on October 23, 1940. Document 2 states that Steve Jobs died on October 5, 2011. Therefore, Pelé was 70 years old when Steve Jobs died.
Answer: 70 years old
